### Install dependencies

In [1]:
%env HDF5_USE_FILE_LOCKING=FALSE
%env WANDB_MODE=dryrun
%env WANDB_NOTEBOOK_NAME=deepmr

In [2]:
import bpnet
from bpnet.cli.contrib import ContribFile
from bpnet.plot.tracks import plot_tracks, to_neg

import os
import uuid
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import clear_output, HTML
from pathlib import Path
import pandas as pd
import numpy as np
clear_output()

#### Optional: Setup wandb

In [3]:
import wandb

wandb.init(project='deepmr', entity='an1lam', mode='disabled')

In [4]:
# config variables
n_reps = 5

# file paths
config_dir = Path('./bpnet/') 

model_config_fname = 'ChIP-nexus-default.gin'
data_config_fname = 'ChIP-nexus.dataspec.yml'

timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
project_dir = '/home/ubuntu/dev/an1lam/deepmr/'
data_dir = os.path.join(project_dir, 'dat')
output_dir = os.path.join(data_dir, f'res-bpnet-training-{timestamp}')
output_dir

'/home/ubuntu/dev/an1lam/deepmr/dat/res-bpnet-training-2021-01-02-04-48-48'

In [5]:
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, 'output_ensemble'), exist_ok=True)

In [6]:
!cat {config_dir}/{data_config_fname}

task_specs:
  Oct4:
    pos_counts: /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Oct4/counts.pos.bw
    neg_counts: /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Oct4/counts.neg.bw
    peaks: /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Oct4/idr-optimal-set.summit.bed.gz
  Sox2:
    pos_counts: /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Sox2/counts.pos.bw
    neg_counts: /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Sox2/counts.neg.bw
    peaks: /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Sox2/idr-optimal-set.summit.bed.gz
  Nanog:
    pos_counts: /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Nanog/counts.pos.bw
    neg_counts: /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Nanog/counts.neg.bw
    peaks: /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Nanog/idr-optimal-

### Data stats

In [7]:
# chromsomome names of differnet peaks
!zcat {project_dir}dat/bpnet-manuscript-data/data/chip-nexus/Sox2/idr-optimal-set.summit.bed.gz \
    | cut -f 1 | sort -u

chr1
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chrX
chrY


Each task (or TF) can specify a set of peaks associated with it. Here are the number of peaks per TF we will use in this tutorial:

In [8]:
tasks = ['Oct4', 'Sox2', 'Nanog', 'Klf4']

# number of peaks per task
for task in tasks:
    print(task)
    !zcat {data_dir}/bpnet-manuscript-data/data/chip-nexus/{task}/idr-optimal-set.summit.bed.gz | wc -l

Oct4
25849
Sox2
10999
Nanog
56459
Klf4
57601


## 2. Train the model

Having specified `dataspec.yml`, we are now ready to train the model with 

```
bpnet train <dataspec.yml> <output dir> [optional flags]`
```


We will use a pre-made model [bpnet9](../bpnet/premade/bpnet9.gin) as a starting point and modify a few parameters specified in the config.gin file. Specifically, we will 
- train the model only on chromosomes 16-19
- evaluate the model on chromosome 2
- use only 3 layers of dilated convolutions 
- use an input sequence length of 200 bp and accordingly lower the augmentation shift to 100 bp

In [9]:
!cat {config_dir}/{model_config_fname} 
# NOTE: test_chr will be also excluded similar to 'exclude_chr'

task_specs:
  Oct4:
    tracks:
    - /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Oct4/counts.pos.bw
    - /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Oct4/counts.neg.bw
    peaks: /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Oct4/idr-optimal-set.summit.bed.gz
  Sox2:
    tracks:
    - /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Sox2/counts.pos.bw
    - /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Sox2/counts.neg.bw
    peaks: /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Sox2/idr-optimal-set.summit.bed.gz
  Nanog:
    tracks:
    - /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Nanog/counts.pos.bw
    - /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Nanog/counts.neg.bw
    peaks: /home/stephenmalina/project/dat/bpnet-manuscript-data/data/chip-nexus/Nanog/idr-optimal-set.summit.bed.gz
  Klf4

Have a look at the original gin file of bpnet9 here: https://github.com/kundajelab/bpnet/blob/master/bpnet/premade/bpnet9-ginspec.gin. For more information on using gin files see <https://github.com/google/gin-config>. 

To track model training and evaluation, we will use [wandb](http://wandb.com/) by adding `--wandb=avsec/bpnet-demo` to `bpnet train`. You can navigate to https://app.wandb.ai/avsec/bpnet-demo to see the training progress.

Let's train!

In [10]:
# setup all the file paths
example_model_dir = os.path.join(output_dir, 'output_ensemble', '0')

In [18]:
# Train for at most 10 epochs
for i in range(1):
    # setup a new run_id (could be done automatically, but then the output directory would change)
    run_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + "_" + str(uuid.uuid4())
    !cd {config_dir} && bpnet train {data_config_fname} --premade=bpnet9 \
        --config={model_config_fname} {output_dir} \
        --run-id '{run_id}' \
        --override='train.epochs=10; train.seed={i}'
    # softlink the new output directory
    !rm -rf {output_dir}/output_ensemble/{i} && ln -srf {output_dir}/{run_id} {output_dir}/output_ensemble/{i}

Using TensorFlow backend.

2021-01-02 05:13:16,856 [WARNING] From /home/ubuntu/anaconda3/envs/deepmr/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

2021-01-02 05:13:17,519 [INFO] NumExpr defaulting to 4 threads.
/home/ubuntu/anaconda3/envs/deepmr/lib/python3.6/site-packages/bpnet/plot/heatmaps.py:6: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid1.colorbar module was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use matplotlib.colorbar instead.
  from mpl_toolkits.axes_grid1.colorbar import colorbar
INFO [01-02 05:13:19] Using gpu: 0, memory fraction: 0.45
2021-01-02 05:13:19.325018: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-01-02 05:13:19.333201: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2021-01-02

/home/ubuntu/anaconda3/envs/deepmr/lib/python3.6/site-packages/bpnet/cli/train.py:208: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  .replace(" = ", ": "))
Epoch 1/10
2021-01-02 05:13:37.962050: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2021-01-02 05:13:38.137972: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2021-01-02 05:13:38.659704: E tensorflow/stream_executor/cuda/cuda_dnn.cc:319] Loaded runtime CuDNN library: 7.5.1 but source was compiled with: 7.6.0.  CuDNN library major and minor version needs to match or have higher minor version in case of CuDNN 7.0 or later version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version 

In [93]:
! echo {output_dir}
! echo
! ls {output_dir}/output_ensemble/0

/home/stephenmalina/project/dat/res-bpnet-training-2020-09-02-16-18-50

bpnet-train.kwargs.json			    history.csv
config.gin				    input-config.gin
config.gin.json				    log
dataspec.yml				    model.h5
evaluate.html				    note_params.json
evaluate.ipynb				    seq_model.pkl
evaluation.valid.json			    wandb.json
events.out.tfevents.1599063546.gregor-3-vm


In [94]:
!ls -latr {example_model_dir}/

total 14976
-rw-r--r--  1 stephenmalina stephenmalina       2 Sep  2 16:18 note_params.json
drwxr-xr-x  2 stephenmalina stephenmalina    4096 Sep  2 16:18 log
-rw-r--r--  1 stephenmalina stephenmalina     328 Sep  2 16:18 wandb.json
-rw-r--r--  1 stephenmalina stephenmalina     302 Sep  2 16:18 input-config.gin
-rw-r--r--  1 stephenmalina stephenmalina     579 Sep  2 16:18 bpnet-train.kwargs.json
-rw-r--r--  1 stephenmalina stephenmalina    1562 Sep  2 16:18 dataspec.yml
-rw-r--r--  1 stephenmalina stephenmalina    5943 Sep  2 16:18 config.gin
-rw-r--r--  1 stephenmalina stephenmalina    4191 Sep  2 16:18 config.gin.json
-rw-r--r--  1 stephenmalina stephenmalina    3743 Sep  2 16:51 history.csv
-rw-r--r--  1 stephenmalina stephenmalina 1733204 Sep  2 16:51 events.out.tfevents.1599063546.gregor-3-vm
-rw-r--r--  1 stephenmalina stephenmalina 1810128 Sep  2 16:52 model.h5
-rw-r--r--  1 stephenmalina stephenmalina 2401402 Sep  2 16:52 seq_model.pkl
-rw-r--r--  1 stephenmalina stephenmalina